In [1]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random
import xlwt
import openpyxl
from openpyxl import Workbook
from pandas import ExcelWriter
from pandas import ExcelFile

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [2]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [3]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-09-24 23:54:27,2,None,None,None,0,AmanteTech,Would next #Apple #iOS update cause your exist...,912102964881367040,...,2133,635,632,869739471570325504,"Sydney, New South Wales",AmanteTech,6314,Pacific Time (US & Canada),0,0
1,1,2017-09-24 23:30:33,0,None,None,None,84,supermeg500,RT @MegsSkunkey: iPhone 8 giveaway!!\nhttps://...,912096951532847104,...,61,361,4512,819370348420284417,,Giveaways~,402,None,0,1
2,2,2017-09-24 22:45:06,0,None,None,None,1860,botlamaya,RT @Sudathchamin: iPhone X new price?Ur left k...,912085514282553344,...,10,617,274,941255144,Colombo,Bot Lamaya [beta],51626,New Delhi,0,1
3,3,2017-09-24 22:45:06,0,None,None,None,1860,hikma_5000,RT @Sudathchamin: iPhone X new price?Ur left k...,912085512776798213,...,1388,143,113,791115735200182272,"Mansfield, TX",Hikma🌹🌹🌹,5432,Pacific Time (US & Canada),0,1
4,4,2017-09-24 22:36:56,0,None,None,None,4,Joey2031,RT @BoinkHunter: Beautiful device. \n#iPhoneX ...,912083456552783873,...,853,66,75,705859071044423680,,Joey,646,None,0,1


In [4]:
df.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,45266.000000,45266.000000,45266.000000,4.526600e+04,45266.000000,4.526600e+04,45266.000000,4.526600e+04,4.526600e+04,45266.000000,45266.000000
mean,49.195864,0.912915,3879.166328,9.146185e+17,8344.810454,9.786278e+03,1840.460346,1.827840e+17,4.709686e+04,0.014514,0.512062
std,28.846285,39.827144,13091.820219,6.425545e+14,23357.029302,1.148350e+05,11001.654680,3.433272e+17,1.319984e+05,0.119599,0.499860
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.210000e+02,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.145397e+17,106.000000,9.800000e+01,130.000000,2.828020e+08,1.526250e+03,0.000000,0.000000
50%,49.000000,0.000000,2.000000,9.145858e+17,1238.000000,3.720000e+02,356.000000,1.630756e+09,8.583500e+03,0.000000,1.000000
75%,74.000000,0.000000,751.000000,9.153291e+17,6862.750000,1.303000e+03,977.000000,4.195543e+09,3.622075e+04,0.000000,1.000000
max,99.000000,8016.000000,142956.000000,9.153658e+17,717217.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [5]:
df.count()

index                      45266
created_at                 45266
favorite_count             45266
in_reply_to_screen_name     3913
in_reply_to_status_id       3276
in_reply_to_user_id         3913
retweet_count              45266
screen_name                45266
text                       45266
tweet_id                   45266
tweet_location                87
user_created_at            45266
user_description           45266
user_favorites_count       45266
user_followers_count       45266
user_friends_count         45266
user_id                    45266
user_location              45266
user_name                  45266
user_statuses_count        45266
user_time_zone             27351
user_verified              45266
RT                         45266
dtype: int64

In [6]:
dfs = df.drop_duplicates(subset = 'tweet_id', inplace=False)

In [7]:
dfs.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,44629.000000,44629.000000,44629.000000,4.462900e+04,44629.000000,4.462900e+04,44629.000000,4.462900e+04,4.462900e+04,44629.000000,44629.000000
mean,49.235027,0.921531,3933.924533,9.146478e+17,8429.744740,9.884793e+03,1844.093594,1.798917e+17,4.741416e+04,0.014542,0.511708
std,28.844561,40.110049,13176.832575,5.903429e+14,23485.935539,1.156411e+05,11060.342246,3.410354e+17,1.321803e+05,0.119712,0.499869
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.210000e+02,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.145416e+17,112.000000,1.000000e+02,132.000000,2.832035e+08,1.569000e+03,0.000000,0.000000
50%,49.000000,0.000000,2.000000,9.145874e+17,1264.000000,3.750000e+02,358.000000,1.617632e+09,8.751000e+03,0.000000,1.000000
75%,74.000000,0.000000,781.000000,9.153297e+17,6990.000000,1.303000e+03,978.000000,4.085374e+09,3.658700e+04,0.000000,1.000000
max,99.000000,8016.000000,142956.000000,9.153658e+17,717217.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [8]:
dfs.count()

index                      44629
created_at                 44629
favorite_count             44629
in_reply_to_screen_name     3912
in_reply_to_status_id       3276
in_reply_to_user_id         3912
retweet_count              44629
screen_name                44629
text                       44629
tweet_id                   44629
tweet_location                82
user_created_at            44629
user_description           44629
user_favorites_count       44629
user_followers_count       44629
user_friends_count         44629
user_id                    44629
user_location              44629
user_name                  44629
user_statuses_count        44629
user_time_zone             27033
user_verified              44629
RT                         44629
dtype: int64

In [9]:
Number_ofWords_Text = dfs.text.str.split().str.len()

In [10]:
Number_ofWords_Text

0        20
1        13
2        17
3        17
4        11
5        11
6        11
7         9
8        13
9        13
10       13
11       13
12       13
13       13
14       15
15       13
16       13
17       17
18       13
19       14
20       19
21       18
22       22
23       12
24       13
25       18
26       18
27       16
28       13
29       13
         ..
45236    18
45237    13
45238    17
45239     6
45240    20
45241    24
45242    13
45243    23
45244    24
45245    13
45246    14
45247     5
45248    24
45249    18
45250    17
45251    23
45252    27
45253    17
45254    11
45255    12
45256    12
45257    23
45258    14
45259    13
45260    25
45261    11
45262    27
45263    11
45264    16
45265     6
Name: text, Length: 44629, dtype: int64

In [11]:
dfs['Number_ofWords_Text']  = Number_ofWords_Text.values

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
dfs.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT,Number_ofWords_Text
count,44629.000000,44629.000000,44629.000000,4.462900e+04,44629.000000,4.462900e+04,44629.000000,4.462900e+04,4.462900e+04,44629.000000,44629.000000,44629.000000
mean,49.235027,0.921531,3933.924533,9.146478e+17,8429.744740,9.884793e+03,1844.093594,1.798917e+17,4.741416e+04,0.014542,0.511708,16.005759
std,28.844561,40.110049,13176.832575,5.903429e+14,23485.935539,1.156411e+05,11060.342246,3.410354e+17,1.321803e+05,0.119712,0.499869,5.552562
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.210000e+02,1.000000e+00,0.000000,0.000000,1.000000
25%,24.000000,0.000000,0.000000,9.145416e+17,112.000000,1.000000e+02,132.000000,2.832035e+08,1.569000e+03,0.000000,0.000000,12.000000
50%,49.000000,0.000000,2.000000,9.145874e+17,1264.000000,3.750000e+02,358.000000,1.617632e+09,8.751000e+03,0.000000,1.000000,16.000000
75%,74.000000,0.000000,781.000000,9.153297e+17,6990.000000,1.303000e+03,978.000000,4.085374e+09,3.658700e+04,0.000000,1.000000,19.000000
max,99.000000,8016.000000,142956.000000,9.153658e+17,717217.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000,39.000000


In [15]:
dfs.to_csv("twitter_dataset_v2.csv", float_format='{:f}'.format)
writer = pd.ExcelWriter('twitter_dataset_v2.xlxs', engine = "openpyxl")
dfs.to_excel(writer, float_format='{:f}'.format, sheet_name='Sheet1')

ValueError: Invalid extension for engine 'openpyxl22': 'xlxs'